# Lab 8: Manual Annotation (PBMC)

## Objectives
- Use canonical markers to assign cluster labels
- Record evidence + confidence
- Produce a final UMAP with cell type labels

## Outputs
- `../results/lab08_manual_annotations.csv`
- `../figures/lab08_umap_celltypes.png`

---


In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt
from pathlib import Path

adata = sc.datasets.pbmc3k_processed()
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=40)
sc.tl.leiden(adata, resolution=0.5, key_added='leiden')
sc.tl.umap(adata)

# Marker dotplot to guide labeling
pbmc_markers = {
    'CD4 T': ['IL7R', 'CCR7'],
    'CD8 T': ['CD8A', 'CD8B'],
    'T': ['CD3D', 'CD3E'],
    'B': ['MS4A1', 'CD79A'],
    'NK': ['NKG7', 'GNLY'],
    'Mono': ['LYZ', 'S100A8'],
    'DC': ['FCER1A', 'CST3'],
    'Platelet': ['PPBP', 'PF4'],
}
sc.pl.dotplot(adata, var_names=pbmc_markers, groupby='leiden', standard_scale='var')

# TODO: Fill this mapping after inspecting marker expression
cluster_to_celltype = {
    # '0': 'CD4 T',
}

adata.obs['celltype_manual'] = adata.obs['leiden'].map(cluster_to_celltype).astype('category')

# Save mapping table
Path('../results').mkdir(exist_ok=True)
Path('../figures').mkdir(exist_ok=True)
out_csv = Path('../results/lab08_manual_annotations.csv')
adata.obs[['leiden', 'celltype_manual']].to_csv(out_csv)
print(f"Wrote {out_csv}")

# Plot (may show NaN if you haven't filled mapping yet)
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
sc.pl.umap(adata, color='celltype_manual', ax=ax, show=False, title='Manual cell type labels')
out_fig = Path('../figures/lab08_umap_celltypes.png')
plt.savefig(out_fig, dpi=150)
print(f"Wrote {out_fig}")
